In [6]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\park2\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=
                             ('headers', 'footers', 'quotes'))
documents = dataset.data
print('샘플의 수 : ', len(documents))

샘플의 수 :  11314


In [8]:
# 텍스트 전처리
# 알파벳을 제외한 구두점, 숫자, 특수문자를 제거하는 것
# 정규 표현식을 통해서 해결 가능
# 짧은 단어는 유용한 정보를 담고 있지 않다고 가정
# 길이가 짧은 단어도 제거
# 모든 알파벳을 소문자로 바꿔서 단어의 개수를 줄이는 작업

news_df = pd.DataFrame({'document' : documents})
# 특수문자 제거
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ", regex=True)

# 길이가 3이하인 단어는 제거 (길이가 짧은 단어 제거)
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x : ' '.join
                                                  ([w for w in x.split() if len(w) > 3]))

# 전체 단어에 대한 소문자 변환
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x : x.lower())


In [9]:
# 뉴스그룹 데이터에서 불용어를 제거
# 불용어를 제거하기 위해 토큰화를 우선 수행
# 토큰화 -> 불용어 제거 순차적으로 진행

# NLTK로부터 불용어를 받아온다
stop_words = stopwords.words('english')
tokenized_doc = news_df['clean_doc'].apply(lambda x : x.split()) # 토큰화
tokenized_doc = tokenized_doc.apply(lambda x : [item for item in x if item not in stop_words])
# 불용어 제거

In [10]:
tokenized_doc[:5]

0    [well, sure, story, seem, biased, disagree, st...
1    [yeah, expect, people, read, actually, accept,...
2    [although, realize, principle, strongest, poin...
3    [notwithstanding, legitimate, fuss, proposal, ...
4    [well, change, scoring, playoff, pool, unfortu...
Name: clean_doc, dtype: object

In [13]:
# 각 단어에 정수 인코딩을 하는 동시에, 각 뉴스에서의 단어의 빈도수 기록
# 각 단어를 (word_id, word_frequency) 형태로 바꿈
# word_id = 단어가 정수 인코딩 된 값
# word_frequency = 해당 뉴스에서의 해당 단어의 빈도수

# gensim의 corpora.Dictionary()를 사용하여 쉽게 구함

from gensim import corpora
dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]
print(corpus[1]) # 수행된 결과에서 두번째 뉴스 출력. 첫번째 문서의 인덱스는 0

[(52, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 2), (67, 1), (68, 1), (69, 1), (70, 1), (71, 2), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 2), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 2), (86, 1), (87, 1), (88, 1), (89, 1)]


In [14]:
# (66,2)는 정수 인코딩이 66으로 할당된 단어가 두 번째 뉴스에서는 2번 등장했음을 의미
# 66이라는 값을 가지는 단어가 정수 인코딩이 되기 전에는 어떤 단어였는지 확인해보자
print(dictionary[66])

faith


In [16]:
# 총 학습된 단어의 개수를 확인
len(dictionary)

64281

In [17]:
# LDA 모델 훈련시키기
# 기존의 뉴스 데이터가 총 20개의 카테고리를 가지고 있었으므로
# Topic의 개수를 20으로 하여 LDA모델을 학습

import gensim
NUM_TOPICS = 20 # 20개의 토픽, k=20

ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics= NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
  print(topic)
  
# 각 단어 앞에 붙은 수치는 단어의 해당 토픽에 대한 기여도를 보여줌
# passes는 알고리즘의 동작 횟수를 뜻함
# 알고리즘이 결정하는 토픽의 값이 적절히 수렴할 수 있도록 적당한 횟수를 정해서 수행
# num_words를 4로 설정하여 4개의 단어만 출력하도록 함

(0, '0.017*"printer" + 0.011*"fonts" + 0.011*"font" + 0.010*"print"')
(1, '0.012*"israel" + 0.010*"jews" + 0.009*"turkish" + 0.007*"israeli"')
(2, '0.011*"would" + 0.010*"people" + 0.007*"think" + 0.006*"jesus"')
(3, '0.052*"space" + 0.016*"nasa" + 0.011*"launch" + 0.011*"earth"')
(4, '0.009*"good" + 0.008*"much" + 0.007*"bike" + 0.007*"like"')
(5, '0.011*"government" + 0.009*"system" + 0.009*"would" + 0.009*"encryption"')
(6, '0.053*"file" + 0.035*"entry" + 0.033*"output" + 0.018*"section"')
(7, '0.010*"would" + 0.010*"drive" + 0.009*"thanks" + 0.009*"know"')
(8, '0.016*"university" + 0.013*"april" + 0.012*"center" + 0.011*"national"')
(9, '0.014*"neutral" + 0.012*"bandwidth" + 0.011*"conductor" + 0.009*"idle"')
(10, '0.008*"rate" + 0.007*"study" + 0.007*"year" + 0.006*"runs"')
(11, '0.012*"people" + 0.006*"armenian" + 0.006*"said" + 0.006*"would"')
(12, '0.025*"game" + 0.023*"team" + 0.018*"games" + 0.016*"play"')
(13, '0.017*"would" + 0.015*"like" + 0.014*"know" + 0.013*"think"')
(1

In [18]:
# 10개의 단어를 출력하고 싶은 경우
print(ldamodel.print_topics())

[(0, '0.017*"printer" + 0.011*"fonts" + 0.011*"font" + 0.010*"print" + 0.007*"laser" + 0.006*"model" + 0.006*"shell" + 0.006*"printing" + 0.005*"polygon" + 0.005*"converted"'), (1, '0.012*"israel" + 0.010*"jews" + 0.009*"turkish" + 0.007*"israeli" + 0.007*"guns" + 0.006*"state" + 0.006*"turkey" + 0.006*"killed" + 0.006*"weapons" + 0.006*"government"'), (2, '0.011*"would" + 0.010*"people" + 0.007*"think" + 0.006*"jesus" + 0.006*"believe" + 0.005*"many" + 0.005*"even" + 0.005*"know" + 0.004*"true" + 0.004*"also"'), (3, '0.052*"space" + 0.016*"nasa" + 0.011*"launch" + 0.011*"earth" + 0.009*"satellite" + 0.008*"shuttle" + 0.008*"orbit" + 0.007*"lunar" + 0.007*"mission" + 0.007*"moon"'), (4, '0.009*"good" + 0.008*"much" + 0.007*"bike" + 0.007*"like" + 0.006*"cars" + 0.005*"engine" + 0.005*"speed" + 0.005*"well" + 0.005*"also" + 0.004*"time"'), (5, '0.011*"government" + 0.009*"system" + 0.009*"would" + 0.009*"encryption" + 0.008*"chip" + 0.008*"security" + 0.007*"keys" + 0.007*"clipper" + 0.

In [19]:
# LDA 시각화 하기
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

# 좌측의 원들은 각각 20개의 topic을 나타냄
# 각 원과의 거리는 각 topic들이 서로 얼마나 다른지를 보여줌

# 주의할 점
# LDA 모델에서는 인덱스 시작이 0부터인데
# 시각화에서는 인덱스 시작이 1부터임 

In [20]:
# 문서 별 토픽 분포 보기

# topic 별 단어 분포는 확인했지만, 문서 별 topic 분포는 확인하지 못함
# 각 문서의 topic 분포는 이미 훈련된 LDA 모델에 전체 데이터가 정수 인코딩 된 결과를 넣은 후에야 확인이 가능하다

# 상위 5개의 문서에 대해서만 topic 분포를 확인
for i, topic_list in enumerate(ldamodel[corpus]):
  if i==5:
    break
  print(i, '번째 문서의 topic 비율은', topic_list)

0 번째 문서의 topic 비율은 [(1, 0.25294408), (2, 0.3906676), (10, 0.24255058), (11, 0.100916974)]
1 번째 문서의 topic 비율은 [(1, 0.028062204), (2, 0.44514465), (13, 0.5044027)]
2 번째 문서의 topic 비율은 [(1, 0.3209692), (2, 0.0783304), (5, 0.04103058), (11, 0.12878062), (13, 0.36112878), (15, 0.04121472), (18, 0.017510917)]
3 번째 문서의 topic 비율은 [(4, 0.16959465), (5, 0.5005455), (7, 0.07248204), (13, 0.13685286), (16, 0.07705666), (18, 0.032337055)]
4 번째 문서의 topic 비율은 [(6, 0.2358245), (12, 0.36055273), (13, 0.3721331)]


In [23]:
# 깔끔한 데이터 프레임 형태로 출력
def make_topictable_per_doc(ldamodel, corpus):
  topic_table = pd.DataFrame()
  
  # 몇 번째 문서인지를 의미하는 문서번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내온다.
  for i, topic_list in enumerate(ldamodel[corpus]):
    doc = topic_list[0] if ldamodel.per_word_topics else topic_list
    doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
    # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬
    # ex) 정렬 전 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (10번 토픽, 5%), (12번 토픽, 21.5%)
    # ex) 정렬 후 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (12번 토픽, 21.5%), (10번 토픽, 5%)
    
    # 모든 문서에 대해서 아래를 수행
    for j, (topic_num, prop_topic) in enumerate(doc): # 몇 번 토픽인지와 비중을 나눠서 저장
      if j==0: # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
        topic_table = topic_table._append(pd.Series([int(topic_num), round(prop_topic,4), topic_list]), ignore_index=True)
        # 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중과, 전체 토픽의 비중을 저장
      else:
        break
    return(topic_table)

In [26]:
topictable = make_topictable_per_doc(ldamodel, corpus)
topictable = topictable.reset_index() # 문서 번호를 의미하는 column로 사용하기 위해서 인덱스 열을 하나 더 생성
topictable.columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topictable[:10]

,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,0,2,0.3907,"[(1, 0.25294515), (2, 0.3906963), (10, 0.24255..."
